# This Notebook contains various code snippets for exploration of the Digital Typhoon Dataset and for exploration of the latent space built using a MoCo trained vision encoder.

In [6]:
import numpy as np
from pyphoon2.DigitalTyphoonDataset import DigitalTyphoonDataset as DTD
from torchvision import transforms as T

from lib.utils.fisheye import FishEye

transforms = T.Compose([
    T.ToTensor(),
    #T.Resize(256),
    FishEye(256, .5),
    #T.RandomApply([T.GaussianBlur(3, [.1, 2.])], p=0.5),
    #T.RandomSolarize(0.6, p=0),
    #T.RandomHorizontalFlip(p=0.5),
    #T.RandomVerticalFlip(p=0.5),
    #T.Normalize(mean=269.15, std=24.14),
])

def transform_func(obj):
    img, labels = obj
    img_range = [150, 350]
    img, labels = obj
    img = (img - img_range[0])/(img_range[1]-img_range[0])

    return transforms(img.astype(np.float32)), labels

prefix="/fs9/datasets/typhoon-202404/wnp"

dataset = DTD(f"{prefix}/image/",
              f"{prefix}/metadata/",
              f"{prefix}/metadata.json",
              get_images_by_sequence=False,
              labels=("pressure", "wind", "lat", "lng", "grade", "interpolated", "mask_1_percent"),
              split_dataset_by="sequence",
              load_data_into_memory="track",
              filter_func= lambda x: (x.year() == 2017 or x.year() == 2018) and x.grade() < 6,
              transform=None,#transform_func,
              ignore_list=[],
              verbose=False)

In [7]:
print(len(dataset))
print(dataset.number_of_nonempty_sequences)

9582
57


In [8]:
from tqdm import tqdm

pressures = []
winds = []
lat = []
lng = []
grades = []
interpolateds = []
for i in tqdm(range(len(dataset))):
    _, labels = dataset[i]
    pressures.append(labels[0])
    winds.append(labels[1])
    lat.append(labels[2])
    lng.append(labels[3])
    grades.append(labels[4])
    interpolateds.append(labels[6])
print(f"Pressure| mean: {np.mean(pressures)} std: {np.std(pressures)}")
print(f"Wind| mean: {np.mean(winds)} std: {np.std(winds)}")
print(f"Lat| mean: {np.mean(lat)} std: {np.std(lat)}")
print(f"Lng| mean: {np.mean(lng)} std: {np.std(lng)}")

100%|██████████| 192956/192956 [00:02<00:00, 79475.50it/s]


Pressure| mean: 983.7059485063952 std: 22.575873184072947
Wind| mean: 36.85347954974191 std: 32.76338099425619
Lat| mean: 22.584546062314725 std: 10.60326799814119
Lng| mean: 136.20306411824458 std: 17.278662751998144


In [4]:
ss = sum(interpolateds)
print(len(dataset) - ss)
print(ss)

-40486886.0
40679842.0


In [13]:
len(dataset) - np.sum(np.array(interpolateds)==0)

2061

In [ ]:
from matplotlib import pyplot as plt

plt.hist(pressures, bins=20,)
plt.xlabel("Central pressure (hPa)")
plt.title("Distribution of central pressures (hPa) through Digital Typhoon Dataset")

In [ ]:
import plotly.express as px

plt.scatter(x=winds, y=grades)
plt.xlabel("10 min sustained wind speed (kt)")
plt.ylabel("Grade")

In [40]:
from pyphoon2.DigitalTyphoonDataset import DigitalTyphoonDataset as DTD

dataset = DTD(image_dir="/fs9/gaspar/data/WP/image/",
              metadata_dir="/fs9/gaspar/data/WP/metadata/",
              metadata_json="/fs9/gaspar/data/WP/metadata.json",
              get_images_by_sequence=False,
              labels=("pressure", "wind", "lat", "lng", "grade"),
              split_dataset_by="sequence",
              load_data_into_memory="track",
              filter_func= lambda x: x.grade() == 6,
              transform=None,#transform_func,
              ignore_list=[],
              verbose=False)

print(len(dataset))

22029


In [4]:
import numpy as np
from tqdm import tqdm

print(f"All sequences: {len(dataset)}")
transitions = []
for i in tqdm(range(len(dataset))):
    _, labels = dataset[i]
    grades = np.unique(labels[:,-1])
    if len(grades) > 1:
        transitions.append(grades)
print(f"Sequences with transition in grade: {len(transitions)}")

All sequences: 1099


100%|██████████| 1099/1099 [00:01<00:00, 657.52it/s]

Sequences with transition in grade: 1097


In [ ]:
et_trans = []
for t in transitions:
    if 6 in t:
        et_trans.append(t)
print(len(et_trans))
print(et_trans)

In [ ]:
import matplotlib.pyplot as plt
import torch

img, label = dataset[15175]
#print(torch.median(img))
plt.title(label)
plt.imshow(img.squeeze(), cmap="grey")

In [ ]:

import torch.nn.functional as F


def get_of_fisheye(height, width, center, magnitude):
  xx, yy = torch.linspace(-1, 1, width), torch.linspace(-1, 1, height)
  gridy, gridx  = torch.meshgrid(yy, xx)   #create identity grid
  grid = torch.stack([gridx, gridy], dim=-1)
  #print(grid)
  d =  center - grid         #calculate the distance(cx - x, cy - y)
  d_sum = torch.sqrt((d**2).sum(axis=-1)) # sqrt((cx-x)**2 + (cy-y)**2)
  #grid = grid.clamp(0,1)
  grid += d * d_sum.unsqueeze(-1) * magnitude #calculate dx & dy and add to original values
  return grid.unsqueeze(0)    #unsqueeze(0) since the grid needs to be 4D.

def fisheye_grid(width, height, cx, cy, k):
    x = torch.linspace(-1, 1, width)
    y = torch.linspace(-1, 1, height)
    xx, yy = torch.meshgrid(x, y)

    # Convert to polar coordinates
    r = torch.sqrt(xx**2 + yy**2)

    theta = torch.atan2(yy,xx)

    # Apply fisheye distortion equation
    r = r**k

    # Convert back to cartesian coordinates
    new_x = cx + r * torch.sin(theta)
    new_y = cy + r * torch.cos(theta)

    # Stack the coordinates
    grid = torch.stack([new_x, new_y], dim=-1)
    grid = torch.clamp(grid, -1, 1)

    return grid

def fisheye(image, cx, cy, k):
    # get image size
    height, width = image.shape[-2:]

    # generate fisheye grid
    grid = fisheye_grid(int(width/2), int(height/2), cx, cy, k)
    grid = grid.unsqueeze(0).to(image.device).float()

    # apply grid sample
    warped_image = F.grid_sample(image.unsqueeze(0), grid,align_corners=True)
    #cropped_image = tv.center_crop(warped_image, height)
    return warped_image.squeeze(0)

def fisheye_transform(img, alpha=0.5):
    """
    Applies a fisheye transformation to the input image.
    
    Args:
        img (torch.Tensor): Input image tensor of shape (B, C, H, W).
        alpha (float): Fisheye transformation parameter (0 < alpha < 1).
    
    Returns:
        torch.Tensor: Transformed image tensor of shape (B, C, H, W).
    """
    B, C, H, W = img.shape

    # Create a grid of normalized coordinates
    x, y = torch.meshgrid(torch.linspace(-1, 1, int(W/2)), torch.linspace(-1, 1, int(H/2)))
    coords = torch.stack((y, x), dim=-1).to(img.device)

    # Apply fisheye transformation to the coordinates
    r = torch.sqrt(coords[:, :, 0]**2 + coords[:, :, 1]**2)
    radial_scale = torch.pow(r, alpha)#(1 - torch.pow(r, alpha)) / r
    radial_scale[r == 0] = 1.0
    fisheye_coords = coords * torch.unsqueeze(radial_scale, -1)

    # Clamp the transformed coordinates to [-1, 1] range
    fisheye_coords = torch.clamp(fisheye_coords, min=-1, max=1)

    # Sample the input image using the transformed coordinates
    fisheye_img = F.grid_sample(img, fisheye_coords.unsqueeze(0).repeat(B, 1, 1, 1), mode="bilinear", align_corners=True)

    return fisheye_img
#fisheye_grid = get_of_fisheye(256, 256, torch.tensor([0,0]),5)
#fisheye_grid = fisheye_grid(256, 256, 128,128, 1.5)

#fisheye_output = F.grid_sample(img.unsqueeze(0), fisheye_grid)
#fisheye_output = fisheye(img, 0, 0, 1.2)
fisheye_output = fisheye_transform(img.unsqueeze(0), alpha=0.5)
plt.title(label)
plt.imshow(fisheye_output.squeeze(), cmap="grey")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

images, labels = dataset[789]
s = len(labels)

fig, axs = plt.subplots(1, 2)

spl = 220#np.random.randint(s)
axs[0].set_title(str(labels[spl]) + f" {spl}/{s}")
axs[0].imshow(images[spl], cmap="grey")
spl = np.min((s-1, spl+np.random.randint(1,4)))
axs[1].set_title(str(labels[spl]) + f" {spl}/{s}")
axs[1].imshow(images[spl], cmap="grey")


In [ ]:
print(dataset[0])

In [ ]:
import random

import numpy as np
from torch.utils.data import DataLoader

from lib.utils.dataset import SequenceTyphoonDataset as STD

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

dataset = STD(labels=["month", "day", "hour", "pressure", "wind", "grade"],
              include_images=False,
              x=[0,1,2,3,4,5],
              y=[3,4,5],
              num_inputs=12,
              num_preds=1)
train, val, test = dataset.random_split([0.7, 0.15, 0.15], split_by="sequence")

print(f"\n{len(train)} train sequences")
print(f"{len(val)} val sequences")
print(f"{len(test)} test sequences")

test_loader = DataLoader(test,
                        batch_size=1,
                        shuffle=False,
                        num_workers=0)


In [ ]:
print(len(dataset))

1094


In [ ]:
inp, preds = dataset[569]
inp.shape


torch.Size([12, 74])

In [1]:
import random

import numpy as np
import torch
from torch.utils.data import DataLoader

from lib.utils.dataset import SequenceTyphoonDataset as STD

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

dataset = STD(labels=["grade"],
              preprocessed_path="vitp_10k_w6",
              latent_dim=384,
              x=[0],
              y=[0],
              num_inputs=1,
              num_preds=1,
              interval=1,
              #filter_func= lambda x: x.grade() < 6,
              output_all=True,
              prefix = "/fs9/datasets/typhoon-202404/wnp")
#dataset = dataset.random_split([1], split_by="sequence")[0]
train, val, test = dataset.random_split([0.7, 0.15, 0.15], split_by="sequence")

train_loader = DataLoader(train,
                        batch_size=1,
                        shuffle=False,
                        num_workers=0)

test_loader = DataLoader(test,
                        batch_size=1,
                        shuffle=False,
                        num_workers=0)



Expected model inputs: 6
Expected model outputs: 6

[782, 167, 167] 1116
1116


In [2]:
import torch
from tqdm import tqdm

train_grades = []
train_pressures = []
train_features = []
train_seq_ids = []
train_pic_ids = []

test_grades = []
test_pressures = []
test_features = []
test_seq_ids = []
test_pic_ids = []

num_features = 512

for i, (features, seq_id) in enumerate(tqdm(test_loader)):
    features = features.squeeze()
    test_grades.append(features[:,dataset.x])
    #test_pressures.append(features[:,dataset.y])
    test_features.append(features[:,-num_features:])
    test_seq_ids.extend([i]*len(features))
    test_pic_ids.extend(list(range(len(features))))

test_grades = torch.argmax(torch.concatenate(test_grades), dim=1) + 2
#test_pressures = torch.concatenate(test_pressures)
test_features = torch.concatenate(test_features)

for i, (features, seq_id)  in enumerate(tqdm(train_loader)):
    features = features.squeeze()
    train_grades.append(features[:,dataset.x])
    #train_pressures.append(features[:,dataset.y])
    train_features.append(features[:,-num_features:])
    train_seq_ids.extend([i]*len(features))
    train_pic_ids.extend(list(range(len(features))))

train_grades = torch.argmax(torch.concatenate(train_grades), dim=1) + 2
#train_pressures = torch.concatenate(train_pressures)
train_features = torch.concatenate(train_features)

print(test_features.shape)
print(train_features.shape)

  0%|          | 0/167 [00:00<?, ?it/s]

100%|██████████| 782/782 [00:03<00:00, 223.79it/s]


torch.Size([28498, 384])
torch.Size([134188, 384])


In [ ]:
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

max_clusters = 10
Ks = range(2, max_clusters)
km = [KMeans(n_clusters=i) for i in Ks]
scores = []

for kmeans in tqdm(km):
    kmeans.fit(train_features)
    scores.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(Ks, scores)
plt.grid(True)
plt.yscale("log")
plt.title("Elbow curve")

In [4]:
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

km = KMeans(n_clusters=10)
train_clusters = km.fit_predict(train_features)


In [ ]:
curr_seq = train_seq_ids[0]
curr_img = train_clusters[0]

seq_transitions = dict()
seq_transitions[train_seq_ids[0]] = []

all_sequences = [[train_clusters[0]]]
all_transitions = []

for i in tqdm(range(1, len(train_clusters))):
    if curr_seq == train_seq_ids[i]:
        all_transitions.append((curr_img, train_clusters[i]))
        seq_transitions[train_seq_ids[i]].append((curr_img, train_clusters[i]))
        all_sequences[-1].append(train_clusters[i])
    else:
        curr_seq = train_seq_ids[i]
        seq_transitions[train_seq_ids[i]] = []
        all_sequences.append([train_clusters[i]])

    curr_img = train_clusters[i]

In [6]:
transition_dict = dict()

for t in all_transitions:
    if t in transition_dict:
        transition_dict[t] += 1
    else:
        transition_dict[t] = 1

In [ ]:
print(len(transition_dict))

print(sorted([(k,v) for k,v in transition_dict.items()], key=lambda x: x[1], reverse=True))

filtered_transitions = [t for t in all_transitions if t[0]!=t[1]]


In [8]:
import networkx as nx

G = nx.DiGraph()

G.add_nodes_from(range(10))
G.add_edges_from(filtered_transitions)

In [ ]:
position = nx.kamada_kawai_layout(G)

nx.draw_networkx(G, pos=position, arrows=True, node_size=50, font_size=5, width=.5)

In [3]:
import umap

# TSNE(2).fit_transform(all_features)
umap_ = umap.UMAP(n_components=2)
umap_results = umap_.fit_transform(train_features)
umap_results_test = umap_.transform(test_features)

/home/gaspar/env/typhoon/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
from sklearn.decomposition import PCA

pca = PCA(2)
pca_results = pca.fit_transform(train_features)
print(sum(pca.explained_variance_ratio_))
print(pca.explained_variance_ratio_)


0.7416075237611591
[0.66488367 0.07672385]


### Umap visualization of the train data, projected to the latent space

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame()
show_test = False

df["x"]=umap_results_test[:, 0] if show_test else umap_results[:, 0]
df["y"]=umap_results_test[:, 1] if show_test else umap_results[:, 1]
df["seq_ids"]=test_seq_ids if show_test else train_seq_ids
df["pic_ids"]=test_pic_ids if show_test else train_pic_ids
df["grades"]=test_grades if show_test else train_grades

hovertemplate= """
<b>Grade</b>: %{customdata[0]}<br>
<b>Seq ID </b>: %{customdata[1]}<br>
<b>Pic ID </b>: %{customdata[2]}<br>
<b>X</b>: %{x:,.2f}
<b>Y</b>: %{y:,.2f}
<extra></extra>
"""
customdata=np.stack((df["grades"], df["seq_ids"], df["pic_ids"]), axis=-1)
fig=px.scatter(df,
           x="x",
           y="y",
           color="grades",
           )#, animation_frame="seq_ids", range_x=[-10,25], range_y=[-10,25])
fig.update_traces(customdata=customdata,
           hovertemplate=hovertemplate)
fig.show()


### Visualizing the evolution of the sequences through time

In [ ]:
fig=px.scatter(df,
           x="x",
           y="y",
           color="grades",
           hover_data="seq_ids",
           animation_frame="pic_ids",
           range_color=[2,7], range_x=[-10,25], range_y=[-10,25])
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0
fig.show()

### Visualizing the whole sequences one by one

In [ ]:
fig=px.scatter(df,
           x="x",
           y="y",
           color="grades",
           hover_data="pic_ids",
           animation_frame="seq_ids",
           range_x=[-10,25],
           range_y=[-10,25],
           range_color=[2,7])

fig.show()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
from numpy.linalg import norm
from sklearn.metrics import pairwise_distances

seq = 72
seq2= 77
seq_ids = np.array(test_seq_ids if show_test else train_seq_ids)
features = test_features[(seq_ids==seq)] if show_test else train_features[(seq_ids==seq)]
features2= test_features[(seq_ids==seq2)] if show_test else train_features[(seq_ids==seq2)]

print(features.shape)
print(features2.shape)
concat = torch.cat((features, features2))
dist = pairwise_distances(concat, metric="cosine", n_jobs=-1)
norms = np.diag(np.log(norm(concat, axis=1)))
norms = pairwise_distances(np.expand_dims(np.log(norm(concat, axis=1)), 1), metric="euclidean", n_jobs=-1)
# TODO something to do with the norms of the feature vectors?x/
px.imshow(dist).show()
px.imshow(norms).show()

In [8]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier

from lib.utils.evaluation import print_confusion_matrix

for grade in range(2,8):
    neigh = KNeighborsClassifier(n_neighbors=11,)
    neigh.fit(X=train_features, y=train_grades==grade)
    predictions = neigh.predict(test_features)
    acc = balanced_accuracy_score(test_grades==grade, predictions)
    print(f"Accuracy for grade {grade} vs all: {acc:.3f}")
    #print_confusion_matrix(predictions, test_grades==grade)

neigh = KNeighborsClassifier(n_neighbors=11,)
neigh.fit(X=train_features, y=train_grades)
predictions = neigh.predict(test_features)
acc = balanced_accuracy_score(test_grades, predictions)
print(f"Accuracy for all: {acc:.3f}")
print_confusion_matrix(predictions, test_grades)


Accuracy for grade 2 vs all: 0.713
Accuracy for grade 3 vs all: 0.573
Accuracy for grade 4 vs all: 0.549
Accuracy for grade 5 vs all: 0.802
Accuracy for grade 6 vs all: 0.897
Accuracy for grade 7 vs all: 0.500
Accuracy for all: 0.464
Confusion Matrix
[[4704 1843  447  470  306    0]
 [1705 2086  869 1016  213    0]
 [ 436 1009  963 1622  172    0]
 [ 283  525  937 5637   26    0]
 [ 282  161   68   29 2671    0]
 [   4    4    2    8    0    0]]


### TODO FORECASTING USING NN CLASSIFICATION/REGRESSION

- NN to find the closest known examples
- From these known examples we see what the passed forecast has been, we follow the typhoon's trajectory in the latent space

### TODO Extra-Tropical storm prediction using latent space boundaries
- Try to find the transition
- SVM?
